In [1]:
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd

# Download WordNet data
nltk.download('wordnet')
nltk.download('omw-1.4') # Extended WordNet data

print("WordNet loaded successfully.")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


WordNet loaded successfully.


In [2]:
def get_word_relations(target_word):
    synonyms = set()
    antonyms = set()
    hypernyms = set()

    # 1. Get all "Synsets" (Synonym Sets) for the word
    # A single word can have multiple meanings (e.g., 'bank' as noun vs verb)
    synsets = wn.synsets(target_word)

    if not synsets:
        print(f"No entry found for '{target_word}' in WordNet.")
        return None

    # We will loop through all meanings (synsets) to gather comprehensive lists
    for syn in synsets:
        # --- A. Synonyms & Antonyms ---
        for lemma in syn.lemmas():
            # Add synonym (the lemma name itself)
            # Replace underscores with spaces for readability
            synonyms.add(lemma.name().replace('_', ' '))

            # Check for antonyms
            if lemma.antonyms():
                for ant in lemma.antonyms():
                    antonyms.add(ant.name().replace('_', ' '))

        # --- B. Hypernyms (Parent Categories) ---
        # We look at the hypernyms of the synset
        for hyper in syn.hypernyms():
            # Get the first lemma name of the hypernym
            hypernyms.add(hyper.lemmas()[0].name().replace('_', ' '))

    # Convert sets to sorted lists for clean display
    return {
        "Word": target_word,
        "Synonyms": list(sorted(synonyms))[:10], # Limit to 10 for readability
        "Antonyms": list(sorted(antonyms))[:10],
        "Hypernyms": list(sorted(hypernyms))[:10]
    }

print("Function defined.")

Function defined.


In [3]:
# List of words to analyze
words_to_analyze = ["good", "dark", "car", "happy", "run"]

results = []

print(f"{'WORD':<10} | {'ANTONYMS':<20} | {'HYPERNYMS (Parent Concept)':<30}")
print("-" * 75)

for word in words_to_analyze:
    data = get_word_relations(word)
    if data:
        # Save for DataFrame later
        results.append(data)

        # Print summary row
        ants = ", ".join(data['Antonyms'][:3]) if data['Antonyms'] else "-"
        hypers = ", ".join(data['Hypernyms'][:3]) if data['Hypernyms'] else "-"
        print(f"{word:<10} | {ants:<20} | {hypers:<30}")

WORD       | ANTONYMS             | HYPERNYMS (Parent Concept)    
---------------------------------------------------------------------------
good       | bad, badness, evil   | advantage, artifact, morality 
dark       | day, light           | condition, illumination, scene
car        | -                    | compartment, motor vehicle, wheeled vehicle
happy      | unhappy              | -                             
run        | idle, malfunction    | accompany, attempt, be        


In [4]:
df_relations = pd.DataFrame(results)

# Display the DataFrame
print("\n--- Detailed WordNet Relations ---")
display(df_relations)


--- Detailed WordNet Relations ---


,Word,Synonyms,Antonyms,Hypernyms
0,good,"[adept, beneficial, commodity, dear, dependabl...","[bad, badness, evil, evilness, ill]","[advantage, artifact, morality, quality]"
1,dark,"[benighted, black, blue, colored, coloured, da...","[day, light]","[condition, illumination, scene, time period, ..."
2,car,"[auto, automobile, cable car, car, elevator ca...",[],"[compartment, motor vehicle, wheeled vehicle]"
3,happy,"[felicitous, glad, happy, well-chosen]",[unhappy],[]
4,run,"[be given, black market, bleed, break away, bu...","[idle, malfunction]","[accompany, attempt, be, become, break, captur..."


In [5]:
def print_hypernym_path(word):
    synsets = wn.synsets(word)
    if not synsets:
        return

    # Pick the first meaning (most common)
    synset = synsets[0]
    print(f"\n--- Concept Hierarchy for '{word}' ({synset.pos()}) ---")
    print(f"Definition: {synset.definition()}")

    # Get the path to root
    # Note: hypernym_paths() returns a list of lists (paths)
    path = synset.hypernym_paths()[0]

    # Print indentation for tree effect
    for level, node in enumerate(path):
        print("  " * level + " -> " + node.name().split('.')[0])

# Test on 'car'
print_hypernym_path("car")


--- Concept Hierarchy for 'car' (n) ---
Definition: a motor vehicle with four wheels; usually propelled by an internal combustion engine
 -> entity
   -> physical_entity
     -> object
       -> whole
         -> artifact
           -> instrumentality
             -> container
               -> wheeled_vehicle
                 -> self-propelled_vehicle
                   -> motor_vehicle
                     -> car
